In [2]:
import agate

In [8]:
exonerations = agate.Table.from_csv('../data/exonerations-20150828.csv')

In [10]:
tester = agate.TypeTester(force={
    'false_evidence': agate.Boolean()
    })

exonerations = agate.Table.from_csv('../data/exonerations-20150828.csv', column_types=tester)

In [11]:
print(exonerations)

|----------------------+---------------|
|  column_names        | column_types  |
|----------------------+---------------|
|  last_name           | Text          |
|  first_name          | Text          |
|  age                 | Number        |
|  race                | Text          |
|  state               | Text          |
|  tags                | Text          |
|  crime               | Text          |
|  sentence            | Text          |
|  convicted           | Number        |
|  exonerated          | Number        |
|  dna                 | Boolean       |
|  dna_essential       | Text          |
|  mistake_witness     | Boolean       |
|  false_confession    | Boolean       |
|  perjury             | Boolean       |
|  false_evidence      | Boolean       |
|  official_misconduct | Boolean       |
|  inadequate_defense  | Boolean       |
|----------------------+---------------|



In [12]:
exonerations.columns['last_name']

In [13]:
exonerations.columns[0]

In [14]:
for row in exonerations.rows: 
    print(row['last_name'])

Abbitt
Abdal
Abernathy
Acero
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Adams
Addison
Aguirre
Ajamu
Aldape Guerra
Aldridge
Alejandro
Alexander
Alexander
Algarin
Allen
Allen
Allen, Jr.
Alowonle
Alvarado
Alvarez
Alvarez
Alvarez
Alvirez
Ambler
Amezquita
Amirault
Amon-Ra
Amrine
Andersen
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Andre
Andrews
Anthony
Antoine
Appling
Arledge
Armstrong
Armstrong
Armstrong
Arrington
Arroyo
Arteaga
Ashe
Atkins
Atkins
Atkins
Atlas
Austin
Austin
Avery
Avery
Avery
Aviles
Awe
Ayers
Ayers
Ayers
Ayers
Baba-Ali
Baddeley
Bailey
Baillie
Bain
Baker
Baker
Baker
Ballard
Ballinger
Baltierrez
Baltrip
Baniani
Banks
Banks
Baran
Barber
Barbour
Barnes
Barnes, Jr.
Barr
Baruxes
Bass
Bates
Batts
Bauer
Baumer
Baylor
Beal
Beaman
Beamon
Beard
Beauchamp
Beaver
Becerra
Beckett
Belcher
Bell
Bell
Bell, Jr.
Bellamy
Berghoff
Beridon
Bermudez
Berry
Berryman
Bianco
Bibbins
bin Wahad
Birden
Bivens
Bivens, Jr.
Bjerklie
Blackshire
Blair
Blake
Bla

In [16]:
num_false_confessions = exonerations.aggregate(agate.Count('false_confession', True))

print(num_false_confessions)

211


In [17]:
median_age = exonerations.aggregate(agate.Median('age'))

print(median_age)

26


/Users/katieshepherd/Dev/data-notebooks/venv/lib/python2.7/site-packages/agate/warns.py:19: NullCalculationWarning: Column "age" contains nulls. These will be excluded from Median calculation.
  ), NullCalculationWarning)


In [18]:
num_without_age = exonerations.aggregate(agate.Count('age', None))

print(num_without_age)

9


In [19]:
with_age = exonerations.where(lambda row: row['age'] is not None)

In [20]:
old = len(exonerations.rows)
new = len(with_age.rows)

print(old-new)

9


In [21]:
median_age = with_age.aggregate(agate.Median('age'))

print(median_age)

26


In [22]:
with_years_in_prison = exonerations.compute([
        ('years_in_prison', agate.Change('convicted', 'exonerated'))
    ])

median_years = with_years_in_prison.aggregate(agate.Median('years_in_prison'))

print(median_years)

8


In [23]:
full_names = exonerations.compute([
        ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row))
    ])

In [25]:
with_computations = exonerations.compute([
    ('full_name', agate.Formula(agate.Text(), lambda row: '%(first_name)s %(last_name)s' % row)),
    ('years_in_prison', agate.Change('convicted', 'exonerated'))
])

In [26]:
sorted_by_age = exonerations.order_by('age')

In [27]:
youngest_ten = sorted_by_age.limit(10)

In [28]:
youngest_ten.print_table(max_columns=7)

|------------+------------+-----+-----------+-------+---------+---------+------|
|  last_name | first_name | age | race      | state | tags    | crime   | ...  |
|------------+------------+-----+-----------+-------+---------+---------+------|
|  Murray    | Lacresha   |  11 | Black     | TX    | CV, F   | Murder  | ...  |
|  Adams     | Johnathan  |  12 | Caucasian | GA    | CV, P   | Murder  | ...  |
|  Harris    | Anthony    |  12 | Black     | OH    | CV      | Murder  | ...  |
|  Edmonds   | Tyler      |  13 | Caucasian | MS    |         | Murder  | ...  |
|  Handley   | Zachary    |  13 | Caucasian | PA    | A, CV   | Arson   | ...  |
|  Jimenez   | Thaddeus   |  13 | Hispanic  | IL    |         | Murder  | ...  |
|  Pacek     | Jerry      |  13 | Caucasian | PA    |         | Murder  | ...  |
|  Barr      | Jonathan   |  14 | Black     | IL    | CDC, CV | Murder  | ...  |
|  Brim      | Dominique  |  14 | Black     | MI    | F       | Assault | ...  |
|  Brown     | Timothy    | 

In [30]:
binned_ages = exonerations.bins('age', 10, 0, 100)
binned_ages.print_bars('age', 'Count', width=50)

age       Count
[10 - 20)   307 ▓░░░░░░░░░░░░░                    
[20 - 30)   718 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   
[30 - 40)   377 ▓░░░░░░░░░░░░░░░░                 
[40 - 50)   176 ▓░░░░░░░                          
[50 - 60)    53 ▓░░                               
[60 - 70)    10 ▓                                 
[80 - 90)     1 ▓                                 
None          9 ▓                                 
                +-------+-------+--------+-------+
                0      200     400      600    800


In [31]:
by_state = exonerations.group_by('state')

In [32]:
state_totals = by_state.aggregate([
        ('count', agate.Count())
    ])

sorted_totals = state_totals.order_by('count', reverse=True)

sorted_totals.print_table(max_rows=5)

|--------+--------|
|  state | count  |
|--------+--------|
|  TX    |   212  |
|  NY    |   202  |
|  CA    |   154  |
|  IL    |   153  |
|  MI    |    60  |
|  ...   |   ...  |
|--------+--------|


In [35]:
with_years_in_prison = exonerations.compute([
        ('years_in_prison', agate.Change('convicted', 'exonerated'))
    ])

state_totals = with_years_in_prison.group_by('state')

medians = state_totals.aggregate([
        ('count', agate.Count()),
        ('median_years_in_prison', agate.Median('years_in_prison'))
    ])

sorted_medians = medians.order_by('median_years_in_prison', reverse=True)

sorted_medians.print_table(max_rows=5)

|--------+-------+-------------------------|
|  state | count | median_years_in_prison  |
|--------+-------+-------------------------|
|  DC    |    15 |                     27  |
|  NE    |     9 |                     20  |
|  ID    |     2 |                     19  |
|  VT    |     1 |                     18  |
|  LA    |    45 |                     16  |
|  ...   |   ... |                    ...  |
|--------+-------+-------------------------|


In [37]:
only_with_age = with_years_in_prison.where(
    lambda r: r['age'] is not None
)

race_groups = only_with_age.group_by('race')

race_and_age_groups = race_groups.group_by(
    lambda r: '%i0s' % (r['age'] // 10),
    key_name = 'age_group'
)

medians = race_and_age_groups.aggregate([
        ('count', agate.Count()), 
        ('median_years_in_prison', agate.Median('years_in_prison'))
    ])

medians = race_and_age_groups.aggregate([
        ('count', agate.Count()),
        ('median_years_in_prison', agate.Median('years_in_prison'))
    ])

sorted_groups = medians.order_by('median_years_in_prison', reverse=True)

sorted_groups.print_table(max_rows=10)

|------------------+-----------+-------+-------------------------|
|  race            | age_group | count | median_years_in_prison  |
|------------------+-----------+-------+-------------------------|
|  Native American | 20s       |     2 |                   21.5  |
|                  | 20s       |     1 |                   19.0  |
|  Native American | 10s       |     2 |                   15.0  |
|  Native American | 30s       |     2 |                   14.5  |
|  Black           | 10s       |   188 |                   14.0  |
|  Black           | 20s       |   358 |                   13.0  |
|  Asian           | 20s       |     4 |                   12.0  |
|  Black           | 30s       |   156 |                   10.0  |
|  Caucasian       | 10s       |    76 |                    8.0  |
|  Caucasian       | 20s       |   255 |                    8.0  |
|  ...             | ...       |   ... |                    ...  |
|------------------+-----------+-------+----------------------